Author: Tim Mocking

Contact: t.r.mocking@amsterdamumc.nl

In [ ]:
# File locations
imputed_path = ''
gt_path = ''
figures_path =  ''

In [ ]:
# Import all relevant packages
import pandas as pd
import numpy as np
from fcsy import DataFrame
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns
from utils import load_data
from scipy.stats import pearsonr
from statistics import mean, median, stdev
from scipy.stats import wasserstein_distance
from sklearn.metrics import rand_score, adjusted_rand_score, confusion_matrix, \
classification_report, mean_squared_error, recall_score, accuracy_score
from statannot import add_stat_annotation
%load_ext autoreload
%autoreload 2
# Set plotting style
plt.style.use('plotstyle.mplstyle')

In [ ]:
data = load_data(gt_path, imputed_path)

In [ ]:
def calculate_statistics(expression_data, bb_markers, rename, marker_setup, 
                         methods=['CyTOFmerge', 'cyCombine', 'CytoBackBone', 'Infinicyt'],
                         sample_col='sample_id'):
    bin_results = []
    results = []
    for method in methods:
        method_data = expression_data[expression_data['method']==method]
        for sample_id in expression_data[sample_col].unique():
            sample_data = method_data[method_data[sample_col]==sample_id]
            # Filter out unimputed data by cyCombine
            sample_data = sample_data[sample_data['cyCombine_NA']!=True]
            if len(sample_data) == 0:
                print('No sample for '+ method + ' ' + sample_id)
                continue
            sample_data = sample_data.rename(columns=rename)
            imputed_data = sample_data[sample_data['imp_state'] == 1]
            gt_data = sample_data[sample_data['imp_state'] == 0]
            
            assert(len(imputed_data) == len(gt_data))
            
            # Loop over all imputed channels
            for channel in rename.values():
                # In CytoBackBone we don't have a "dataset" component
                if method != 'CytoBackBone':
                    imputed_ds = imputed_data[imputed_data['dataset']==marker_setup[channel]]
                    gt_ds = gt_data[gt_data['dataset']==marker_setup[channel]]
                else:
                    imputed_ds = imputed_data
                    gt_ds = gt_data
                # Calculate pearson correlation, earth mover's distance
                if len(imputed_ds[channel].unique()) == 1:
                    imputed_ds = imputed_ds.reset_index(drop=True)
                    imputed_ds.loc[0,channel] = 0.00001
                cor = pearsonr(imputed_ds[channel], gt_ds[channel])[0]
                RMSD = mean_squared_error(imputed_ds[channel], gt_ds[channel], squared=False)

                # Calculate statistics per quantile
                if method == 'CyTOFmerge':
                    bin_dict = {'Method':method, 'Channel':channel, 'Sample':sample_id}
                    bins = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
                    for i in range(len(bins)):
                        if i != len(bins)-1:
                            print(bins[i], bins[i+1])
                            lower = gt_ds[channel].quantile(bins[i])
                            upper = gt_ds[channel].quantile(bins[i+1])
                            gt_bin = gt_ds[(gt_ds[channel] >= lower) & (gt_ds[channel] < upper)]
                            imp_bin = imputed_ds[imputed_ds['original_ID'].isin(list(gt_bin['original_ID']))]
                            bin_cor = pearsonr(gt_bin[channel], imp_bin[channel])[0]
                            bin_dict[str(bins[i])+' - '+str(bins[i+1])] = bin_cor
                    bin_results.append(bin_dict)
                ymin = np.percentile(gt_ds[channel], 0.1)
                if ymin == 0:
                    ymin = 0.01
                ymax = np.percentile(gt_ds[channel], 99.9)
                NRMSD = RMSD / (ymax - ymin)
                EMD = wasserstein_distance(imputed_ds[channel], gt_ds[channel])
                cors = None
                results.append({'Method':method,
                                'Channel':channel, 
                                'r':cor,
                                'EMD':EMD, 
                                'RMSD': RMSD,
                                'NRMSD': NRMSD,
                                'Sample':sample_id,
                                'n_cells':len(gt_data)})
    results = pd.DataFrame(results)
    return results, bin_results

In [ ]:
flow_variable1 = ['FITC-A', 'APC-A', 'BV605-A', 'BV786-A']
flow_variable2 = ['PE-A', 'PE-CF594-A', 'BV711-A', 'PC7-A']
flow_bb = ["HV500c-A", "BUV395-A", "PerCP-Cy5-5-A", "BUV737-A", "BUV496-A", "BV421-A", "APC-R700-A"]

# Define how the different channels should be renamed in plots
flow_rename = {'APC-A':'KLRG1',
               'BV711-A':'TIM-3',
               'FITC-A':'CD57',
               'BV786-A':'CD27',
               'PE-A':'CD28',
               'PE-CF594-A':'CD95',
               'PC7-A':'TIGIT',
               'BV605-A':'PD-1'}

# Define which markers are imputed in which dataset
flow_marker_setup = {'CD57':2,
                     'KLRG1':2,
                     'PD-1':2,
                     'CD27':2,
                     'CD28':1,
                     'CD95':1,
                     'TIM-3':1,
                     'TIGIT':1}

flow_statistics, flow_bins = calculate_statistics(data, flow_bb, flow_rename, flow_marker_setup)

# Supplemental Figure 5

In [ ]:
temp = pd.DataFrame(flow_bins)
temp = temp.rename(columns={'Channel':'Marker'})
melt = pd.melt(temp, id_vars=['Method', 'Marker', 'Sample'])
fig = plt.figure(figsize=(3, 4))
sns.lineplot(data=melt[melt['Marker'].isin(['CD95', 'CD57', 'PD-1'])],
                       x='variable', y='value', hue='Marker')

# sns.scatterplot(data=melt, x='variable', y='value')
plt.xlabel('Quantile bin')
plt.ylabel('Pearson correlation')
plt.xticks(rotation=45)

plt.savefig(figures_path + 'Supplemental Figure 5.png', dpi=300, bbox_inches='tight')
plt.savefig(figures_path + 'Supplemental Figure 5.tiff', dpi=300, bbox_inches='tight')

# Figure 2

In [ ]:
fig = plt.figure(figsize=(10, 4))

gs = fig.add_gridspec(nrows=1, ncols=2, width_ratios=[0.8, 0.2])

order = ['Infinicyt', 'CyTOFmerge', 'CytoBackBone', 'cyCombine']
palette = ['#f47f2a', '#009cb4', '#ee1d24', '#694893']

ax = fig.add_subplot(gs[0, 0])
sns.barplot(data=flow_statistics, x='Channel', y='EMD', hue='Method', hue_order=order,
            palette=palette, capsize=.05, errwidth=1.25, saturation=1, 
            estimator=median, errorbar=('pi', 50), ax=ax)
ax.set_xlabel('')
ax.legend([], frameon=False)
ax.set_ylabel("Earth mover's distance", fontsize=12)
ax.set_yscale('log')

ax.legend(bbox_to_anchor=(1.75, 0.7), ncol=1, fontsize=12)

ax = fig.add_subplot(gs[0, 1])
sns.boxplot(data=flow_statistics, x='Method', y='EMD', order=order,
            palette=palette, saturation=1, ax=ax)
ax.set_xlabel('')
ax.set_xticklabels([])
ax.legend([], frameon=False)
ax.set_yscale('log')
ax.set_ylabel("Earth mover's distance", fontsize=12)
# add_stat_annotation(ax, data=flow_statistics, x='Method', y="EMD", order=order,
#                     box_pairs=[("Infinicyt", "CyTOFmerge")],
#                     test='t-test_paired', text_format='star', verbose=2, loc='outside')

plt.subplots_adjust(wspace=0.2, hspace=0)

plt.savefig(figures_path + 'Figure 2.png', dpi=300, bbox_inches='tight')
plt.savefig(figures_path + 'Figure 2.tiff', dpi=300, bbox_inches='tight')

# Figure 5

In [ ]:
fig = plt.figure(figsize=(10, 4))

gs = fig.add_gridspec(nrows=1, ncols=2, width_ratios=[0.8, 0.2])

order = ['Infinicyt', 'CyTOFmerge', 'CytoBackBone', 'cyCombine']
palette = ['#f47f2a', '#009cb4', '#ee1d24', '#694893']

ax = fig.add_subplot(gs[0, 0])
sns.barplot(data=flow_statistics, x='Channel', y='r', hue='Method', hue_order=order,
            palette=palette, capsize=.05, errwidth=1.25, saturation=1, 
            estimator=median, errorbar=('pi', 50), ax=ax)
ax.set_xlabel('')
ax.legend([], frameon=False)
ax.set_ylabel("Pearson correlation", fontsize=12)

ax.legend(bbox_to_anchor=(1.75, 0.7), ncol=1, fontsize=12)

ax = fig.add_subplot(gs[0, 1])
sns.boxplot(data=flow_statistics, x='Method', y='r', order=order,
            palette=palette, saturation=1, ax=ax)
ax.set_xlabel('')
ax.set_xticklabels([])
ax.legend([], frameon=False)
ax.set_ylabel("Pearson correlation", fontsize=12)
# add_stat_annotation(ax, data=flow_statistics, x='Method', y="NRMSD", order=order,
#                     box_pairs=[("Infinicyt", "CyTOFmerge"),
#                                ("CyTOFmerge", "CytoBackBone"),
#                                ("CytoBackBone", "cyCombine"),
#                                ("cyCombine", "Infinicyt"),
#                                ("Infinicyt", "CytoBackBone"),
#                                ("cyCombine", "CyTOFmerge")],
#                     test='t-test_paired', text_format='star', verbose=2)
plt.subplots_adjust(wspace=0.2, hspace=0)

plt.savefig(figures_path + 'Figure 5.png', dpi=300, bbox_inches='tight')
plt.savefig(figures_path + 'Figure 5.tiff', dpi=300, bbox_inches='tight')

# Supplemental Figure 6

In [ ]:
fig = plt.figure(figsize=(10, 4))

gs = fig.add_gridspec(nrows=1, ncols=2, width_ratios=[0.8, 0.2])

order = ['Infinicyt', 'CyTOFmerge', 'CytoBackBone', 'cyCombine']
palette = ['#f47f2a', '#009cb4', '#ee1d24', '#694893']

ax = fig.add_subplot(gs[0, 0])
sns.barplot(data=flow_statistics, x='Channel', y='NRMSD', hue='Method', hue_order=order,
            palette=palette, capsize=.05, errwidth=1.25, saturation=1, 
            estimator=median, errorbar=('pi', 50), ax=ax)
ax.set_xlabel('')
ax.legend([], frameon=False)
ax.set_ylabel("NRMSD", fontsize=12)

ax.legend(bbox_to_anchor=(1.75, 0.7), ncol=1, fontsize=12)

ax = fig.add_subplot(gs[0, 1])
sns.boxplot(data=flow_statistics, x='Method', y='NRMSD', order=order,
            palette=palette, saturation=1, ax=ax)
ax.set_xlabel('')
ax.set_xticklabels([])
ax.legend([], frameon=False)
ax.set_ylabel("NRMSD", fontsize=12)
# add_stat_annotation(ax, data=flow_statistics, x='Method', y="NRMSD", order=order,
#                     box_pairs=[("Infinicyt", "CyTOFmerge"),
#                                ("CyTOFmerge", "CytoBackBone"),
#                                ("CytoBackBone", "cyCombine"),
#                                ("cyCombine", "Infinicyt"),
#                                ("Infinicyt", "CytoBackBone"),
#                                ("cyCombine", "CyTOFmerge")],
#                     test='t-test_paired', text_format='star', verbose=2)
plt.subplots_adjust(wspace=0.2, hspace=0)

plt.savefig(figures_path + 'Supplemental Figure 6.png', dpi=300, bbox_inches='tight')
plt.savefig(figures_path + 'Supplemental Figure 6.tiff', dpi=300, bbox_inches='tight')

In [ ]:
flow_statistics.groupby(['Channel']).mean()

In [ ]:
flow_statistics.groupby(['Channel']).std()

In [ ]:
flow_statistics.groupby(['Method']).mean()

In [ ]:
flow_statistics.groupby(['Method']).std()

In [ ]:
flow_statistics.groupby(['Method', 'Channel']).mean().sort_values('r')